In [1]:
cd ~/Desktop/DeepLearningPilot/

/Users/jz-rolling/Desktop/DeepLearningPilot


In [142]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from skimage import filters,morphology,measure
from scipy.stats import pearsonr
import seaborn as sns
from ncolor import label
from timeit import default_timer
import pickle as pk, glob, os
import momia2 as mo
from ncolor import label
import tifffile
from matplotlib import cm
import cv2

In [137]:
sample_dict = {}
for i in range(1,6):
    for j,k in enumerate(sorted(glob.glob('/Volumes/JZSSD_temp/20190709_NQTF/pred_results/{}_*tracker.pk'.format(i)))):
        header = k.split('_tracker.pk')[0]
        phase = np.load(header+'_clips.npy')
        masks = np.load(header+'_omni_masks.npy')
        fitc = np.load(header+'_fitc.npy')
        merged_rp = pk.load(open(k,'rb'))
        sample_dict['{}_{}'.format(i,j)] = [phase,fitc,merged_rp,k,masks]

In [140]:
k = '2_6'
merged_rp = sample_dict[k][2]
phase = sample_dict[k][0]
fitc = sample_dict[k][1]
folder = '/Volumes/JZSSD_temp/20190709_NQTF/trace_plot/EMB/{}/'.format(k)
if not os.path.isdir(folder):
    os.mkdir(folder)

unique_cells = merged_rp['cell_lineage'].unique()
count_dict = {}
for x in unique_cells:
    rid = x.split('.')[0]
    if not rid in count_dict:
        count_dict[rid]=1
    else:
        count_dict[rid]+=1
#filter count
color_count = 0
color_dict = {}
for i,(ac,count) in enumerate(count_dict.items()):
    if count>2:
        color_dict[ac]=cm.get_cmap('tab20')(color_count)
        color_count+=1

for t in merged_rp['$time'].unique():
    #img=label(tracker_msm.frames[t].labeled_mask)
    img = phase[t]
    fl = fitc[t]
    h,w = img.shape
    fig,axes=plt.subplots(figsize=(w/25,h/50),ncols=2,gridspec_kw={'wspace':0})
    
    axes[0].imshow(img,cmap='gist_gray',vmax=1)
    axes[1].imshow(fl,cmap='gist_gray',vmin=550,vmax=1800)
    subset = merged_rp[merged_rp['$time']==t].copy()
    for l in subset['cell_lineage'].unique():
        anc = l.split('.')[0]
        if anc in color_dict:
            color=color_dict[anc]
            cell = subset[subset['cell_lineage']==l].copy()
            for c in cell.index:
                x1,y1,contours,touching_edge = cell.loc[c,['$opt-x1','$opt-y1','$refined_contours','$touching_edge']].values
                if len(contours)>0:
                    if len(contours[0])>10:
                        #contours = [mo.utils.contour.contour_moving_window_smoothing(contours[0])]
                        #simp_contour=mo.core.patch.approximate_polygon(contour,tolerance=0.5)
                        xc,yc = contours[0].T
                        xc = xc+x1
                        yc = yc+y1
                        axes[0].fill(yc,xc,fc=color,ec=None,alpha=0.4)
                        axes[0].plot(yc,xc,color=color,lw=1.5,alpha=1)
                        axes[1].plot(yc,xc,color=color,lw=1.5,alpha=1)
    axes[0].axis('off')
    axes[1].axis('off')
    axes[0].text(0.05,0.95,'{} hours'.format(round(t*0.25,2)),transform=axes[0].transAxes,
                 ha='left',va='top',zorder=3,color='w',fontsize=18,fontweight='bold',fontname='Arial')
    axes[0].plot([(0.05*w),(0.05*w)+(5/0.065)],[0.9*h,0.9*h],color='w',lw=8)
    plt.savefig('{}{}.png'.format(folder,t),bbox_inches='tight')
    plt.close()

In [155]:
# render movies
f = '/Volumes/JZSSD_temp/20190709_NQTF/trace_plot/RIF/11_6/'
header = f.split('/')[-2]
frames = []

for i in range(0,90):
    p = f+'{}.png'.format(i)
    img = cv2.imread(p)
    height, width, _c = img.shape
    frames.append(img)
fourcc = cv2.VideoWriter_fourcc('M','J','P','G')
out2 = cv2.VideoWriter('/Volumes/JZSSD_temp/20190709_NQTF/trace_plot/{}.avi'.format(header),
                        fourcc, fps=6, frameSize=np.flip(img.shape[:2])) 
for i in range(len(frames)):
    out2.write(frames[i])
out2.release()